In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import os
import math
global year
year=[]

In [2]:
def get_corp_code():
    url = "http://comp.fnguide.com/XML/Market/CompanyList.txt"
    resp = requests.get(url)
    resp.encoding = "utf-8-sig"
    data = resp.json()
    comp = data['Co']
    df = pd.DataFrame(data=comp)
    cond = df['gb'] == '701'
    df2 = df[cond].copy()
    return df2

In [3]:
def get_closing_accounts_day(code):
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{code}"
    selector = "#compBody > div.section.ul_corpinfo > div.corp_group1 >  h2:nth-child(4)"
    resp = requests.get(url)
    html = resp.text 
    soup = BeautifulSoup(html, "html5lib")
    tags = soup.select(selector)
    return tags[0].text
get_closing_accounts_day('005930')

'12월 결산'

In [4]:
def get_gpa(code):
    url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"
    gp=[]
    asset=[]
    result=[]
    try: 
        dfs = pd.read_html(url)
        # gross profit
        df = dfs[0]
        df.set_index(df.columns[0],inplace=True)
        tem3 = df.filter(regex=".*12$").columns.tolist()
        for i in range(len(tem3)):
            year.append(tem3[i][2:4])
            
        tem = df.filter(regex=".*12$").filter(like='매출총이익', axis=0)
        for i in range(0,tem.shape[1]):
            if math.isnan(tem.iloc[0,i]):
                gp.append(np.nan)
            else:
                gp.append(float(tem.iloc[0,i])*100000000)
            
        # asset 
        df3 = dfs[2]
        df4 = df3.set_index(df3.columns[0])
        tem2 = df4.filter(regex=".*12$").filter(like='자산', axis=0)
        
        for i in range(0,tem2.shape[1]):
            if math.isnan(tem2.iloc[0,i]):
                asset.append(np.nan)
            else:
                asset.append(float(tem2.iloc[0,i])*100000000)
        #GP/A
        for i in range(len(gp)):
            result.append(gp[i] / asset[i])
    except:
        result = np.nan

    return result
get_gpa('005930')

[0.24407717198561638, 0.26532553937778997, 0.25018615173791797]

In [5]:
def get_cap(code):
    url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"
    try:
        dfs = pd.read_html(url)
        cap = float(dfs[0].iloc[4, 1]) 
        cap = cap * 100000000 
    except:
        cap = np.nan
    return cap
get_cap('005930')

405945200000000.0

In [6]:
def get_pbr(code):
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{code}"
    pbr=[]
    try:
        dfs = pd.read_html(url)
        df5 = dfs[10]
        df6 = df5.set_index(df5.columns[0])
        tem = df6["Annual"].filter(regex=".*12$").filter(like="PBR", axis=0)
        for i in range(0,tem.shape[1]):
            if tem.iloc[0,i]=='':
                pbr.append(np.nan)
            else:
                pbr.append(float(tem.iloc[0,i])*100000000)  
    except:
        pbr = 0
    return pbr
get_pbr('005930')

[206000000.0, 180000000.0, 109000000.00000001]

In [7]:
if not os.path.exists(f"./data/scraping data2"):
    os.makedirs(f"./data/scraping data2")


if __name__ == "__main__":
    df = get_corp_code()
    columns = ['code', 'name', 'day', 'cap']
    data = []
    c=0
    
    for i in range(len(df)):
        data_tem=[]
        s = df.iloc[i]
        acode = s['cd']
        code = acode[1:]
        name = s['nm']
        
        day = get_closing_accounts_day(code)
        cap = get_cap(code)
        gpa = get_gpa(code) 
        pbr = get_pbr(code) 
        
        data_tem.append(code)
        data_tem.append(name)
        data_tem.append(day)
        data_tem.append(cap)
        
        #gpa, pbr
        try:
            for j in range(0,len(year)):
                data_tem.append(gpa[j])
                if c==0:
                    columns.append('gp/a'+year[j])

            for j in range(0,len(year)):
                data_tem.append(pbr[j])
                if c==0:
                    columns.append('pbr'+year[j])
                    year_backup=year
            if len(year)!=0:
                c=1
        except:
            ()
        data.append((data_tem))
        print(i,code,columns,year)
        year=[]
        if (i+1) % 100 == 0:
            df1 = pd.DataFrame(data=data, columns=columns)
            df1.to_excel(f"./data/scraping data2/data_{i+1}.xlsx")
            data = []
        
        time.sleep(0.00001)

    # last companies            
    df1 = pd.DataFrame(data=data, columns=columns)
    df1.to_excel(f"./data/scraping data2/data_last.xlsx")

0 060310 ['code', 'name', 'day', 'cap'] ['20', '21', '22']
1 095570 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
2 006840 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
3 054620 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
4 265520 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
5 211270 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
6 027410 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
7 282330 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']
8 126600 ['code', 'name', 'day', 'cap', 'gp/a20', 'gp/a21', 'gp/a22', 'pbr20', 'pbr21', 'pbr22'] ['20', '21', '22']



KeyboardInterrupt



In [8]:
year=year_backup
# read excels and concat dataframe
xls=os.listdir(f"data/scraping data2")
dfs=[]
for i in xls:
# dtype={"code":str} : sets the "code" column to be of type string (str)
    df=pd.read_excel(f"data/scraping data2/{i}",index_col=0, dtype={"code":str})
    dfs.append(df)
df=pd.concat(dfs)    

In [9]:
!pip install finance-datareader

In [10]:
#Bottom 20% by market cap
index=int(df.shape[0]*0.2)
cap_bound=df.sort_values(by='cap').iloc[index-1]['cap']
cap_bound

54300000000.0

In [11]:
import FinanceDataReader as fdr
cumulative_return=1

for i in range(0,len(year)-1):
    df=pd.concat(dfs)
    index=int(df.shape[0]*0.2)
    cap_bound=df.sort_values(by='cap').iloc[index-1]['cap']
    cond=df['day']=="12월 결산"
    df=df[cond]
    cond=(df['gp/a'+year[i]]>0)&(df['pbr'+year[i]]>0)
    df2=df[cond].copy()
    df2['rank1']=df2['gp/a'+year[i]].rank(ascending=False)
    df2['rank2']=df2['pbr'+year[i]].rank()
    df2['rank']=df2['rank1']+df2['rank2']
    df3=df2.sort_values(by='rank')
    cond=df3['cap']<cap_bound
    df3=df3[cond].head(n=30)
    Return_list=[]
    for j in range(0,df3.shape[0]):
        try:
            df=fdr.DataReader(symbol=df3.iloc[j,0])
            df1=df.loc[df.index.year == int('20'+year[i+1])]
            Return=(df1.iloc[-1]['Close']-df1.iloc[0]['Close'])/df1.iloc[0]['Close']
            Return_list.append(Return)
        except:
            ()
    print('return(',int(year[i+1]),') :',sum(Return_list)/len(Return_list))
    cumulative_return*=1+sum(Return_list)/len(Return_list)
print('New_magic formula_cumulative_return :',cumulative_return-1)
print('New_magic formula_return(CAGR) :',cumulative_return**(1/3)-1)

return( 21 ) : 0.9070092845409441
return( 22 ) : -0.22608520215193958
New_magic formula_cumulative_return : 0.4758627049398789
New_magic formula_return(CAGR) : 0.1385409426964952


In [12]:
df=fdr.DataReader(symbol='229200')
cumulative_kosdaq_return=1
for i in range(0,len(year)-1):
    df1=df.loc[df.index.year == int('20'+year[i+1])]
    kosdaq_return=(df1.iloc[-1]['Close']-df1.iloc[0]['Close'])/df1.iloc[0]['Close']
    print("kosdaq 150 return(",int(year[i+1]),') :',kosdaq_return)
    cumulative_kosdaq_return*=kosdaq_return+1
print('cumulative_kosdaq_return :',cumulative_kosdaq_return-1)  
print('kosdaq_return(CAGR) :',cumulative_kosdaq_return**(1/3)-1  )

kosdaq 150 return( 21 ) : -0.01185194994371979
kosdaq 150 return( 22 ) : -0.35513767629281395
cumulative_kosdaq_return : -0.3627805522739824
kosdaq_return(CAGR) : -0.13947596039798738
